In [100]:
if __name__=='__main__':
    import notebookimport

In [101]:
import os
import sqlite3
import pandas as pd

In [102]:
if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='france18.db'
    conn = sqlite3.connect(dbname)
    rally='France'

In [103]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [104]:
if __name__=='__main__':
    q='SELECT * FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid'
    dd=pd.read_sql(q,conn)
    dd.head()

In [105]:
if __name__=='__main__':
    dd[dd['name']=='RC1']

In [106]:
if __name__=='__main__':
    dd.columns

In [107]:
rules = {'origin': "#driver# and #codriver# #ffo# #entrant#.",
         'ffo':['for the', 'from the', 'of the'],
         'driver': "#driverfullName# (#drivercountryiso3#)",
         'codriver':[ "#codriverfullName# (#codrivercountryiso3#)", "co-driver #codriverfullName# (#codrivercountryiso3#)"],
         'car':"#manufacturername# #vehicleModel#",
        'entrant':"#entrantname#"}

In [108]:
dd.columns=[c.replace('.','') for c in dd.columns]
dd['report'] = dd.apply(lambda row: pandas_row_mapper(row, rules, "#origin#"), axis=1)


In [109]:
classes = pd.api.types.CategoricalDtype(categories = ['RC1', 'RGT', 'RC2', 'RC3', 'RC4','NRC1', 'NRC2','NRC3','NRC4'], 
                                            ordered = True)

In [110]:
basepath = 'report'
if not os.path.exists(basepath):
    os.makedirs(basepath)
    
SUMMARY ='''# Summary

* [Entry List](entry_list.md)
'''

dd['name'] = dd['name'].astype(classes)

grouped = dd.groupby('name')

GROUP=''
for name, group in grouped:
    if len(group):
        GROUP = '## Group {}:\n'.format(name)
        #print('Group {}:'.format(name))
        entries=[]
        for i,r in group.iterrows():
            #print(r['report'])
            entries.append('{}'.format(r['report']))
        GROUP = '{}\n\n- {}'.format(GROUP,'\n- '.join(entries))
        
        fn_g = 'entries_{}.md'.format(name)
        with open('{}/{}'.format(basepath,fn_g), 'w') as out_file:
            out_file.write(GROUP)
        
        SUMMARY = '{s}\n  - [{g}]({f})'.format(s=SUMMARY,g=name, f=fn_g)

with open('{}/SUMMARY.md'.format(basepath), 'w') as out_file:
    out_file.write(SUMMARY)

In [111]:
print(SUMMARY)

# Summary

* [Entry List](entry_list.md)

  - [RC1](entries_RC1.md)
  - [RGT](entries_RGT.md)
  - [RC2](entries_RC2.md)
  - [RC3](entries_RC3.md)
  - [RC4](entries_RC4.md)
